In [2]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras import models
import base64

In [3]:
def predictBase64(listString, path):
    answer = []
    classes=[]
    imgPredict = []
    for _,dirs,_ in os.walk(path + "\\data"):
        classes=dirs
        break
    classes = sorted(classes)
    print(classes)
    for i in listString:
        binaryImg = i.encode('ascii')
        with open("imagePredict.png", "wb") as fh:
            fh.write(base64.decodebytes(binaryImg))
        imgPre = cv2.imread("imagePredict.png")
        imgPre = cv2.cvtColor(imgPre, cv2.COLOR_BGR2RGB)
        print(imgPre.shape)
        imgPredict.append(imgPre)
    model = models.load_model(path + "\\model")
    imgPredict = np.array(imgPredict)
    encodePre = np.argmax(model.predict(imgPredict), axis=-1)
    print(encodePre)
    for i in encodePre:
        answer.append(classes[i]) 
    return answer


In [4]:
testring = "..."

In [5]:
checkstring = "..."

In [6]:
listString = []
listString.append(testring)
listString.append(checkstring)

In [7]:
answer = predictBase64(listString, os.getcwd())
answer

['chech', 'goc', 'te']
(480, 640, 3)
(480, 640, 3)
1/1 [==============================] - 0s 305ms/step
[0 2]


['chech', 'te']